In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import scipy.io as sio
import json
import numpy as np

import sys
sys.path.append('../src/')
import cv2

In [3]:
mat_root = "../src/matlab"

In [4]:
mat_classes = os.path.join(mat_root, "class.mat")
mat_classes = sio.loadmat(mat_classes, simplify_cells=True)['class']
mat_classes

[{'name': 'ADME', 'id': 1},
 {'name': 'AMAN', 'id': 2},
 {'name': 'BEBE', 'id': 3},
 {'name': 'COMP', 'id': 4},
 {'name': 'DADO', 'id': 5},
 {'name': 'DEPR', 'id': 6},
 {'name': 'ELAS', 'id': 7},
 {'name': 'FANT', 'id': 8},
 {'name': 'IMPO', 'id': 9},
 {'name': 'MART', 'id': 10},
 {'name': 'MIGR', 'id': 11},
 {'name': 'NACN', 'id': 12},
 {'name': 'NEVE', 'id': 13},
 {'name': 'NOTC', 'id': 14},
 {'name': 'ORQT', 'id': 15},
 {'name': 'PACN', 'id': 16},
 {'name': 'PARD', 'id': 17},
 {'name': 'REDC', 'id': 18},
 {'name': 'RLAM', 'id': 19},
 {'name': 'SURF', 'id': 20},
 {'name': 'TELE', 'id': 21},
 {'name': 'TRAM', 'id': 22},
 {'name': 'VAPR', 'id': 23},
 {'name': 'VELZ', 'id': 24}]

In [5]:
# lets create a dict
classes = dict()
for class_ in mat_classes:
    
    classes[class_['name']] = class_['id']
classes

{'ADME': 1,
 'AMAN': 2,
 'BEBE': 3,
 'COMP': 4,
 'DADO': 5,
 'DEPR': 6,
 'ELAS': 7,
 'FANT': 8,
 'IMPO': 9,
 'MART': 10,
 'MIGR': 11,
 'NACN': 12,
 'NEVE': 13,
 'NOTC': 14,
 'ORQT': 15,
 'PACN': 16,
 'PARD': 17,
 'REDC': 18,
 'RLAM': 19,
 'SURF': 20,
 'TELE': 21,
 'TRAM': 22,
 'VAPR': 23,
 'VELZ': 24}

much better!!!

In [6]:
# get the indexes of database with 12 words
mat_data = os.path.join(mat_root, "database12_100x100.mat")
mat_data = sio.loadmat(mat_data, simplify_cells=True)
database = mat_data['database']

l = []
for data in database:
    l.append(data['label'])
    
l = np.array(l)    
index12 = np.unique(l)
index12

array([ 1,  6,  8, 10, 12, 13, 16, 17, 18, 19, 22, 23])

In [7]:
# lets check how the data is stored in .mat
mat_data['database']

[{'sample': array([[0.02090083, 0.02097525, 0.02082742, ..., 0.02071016, 0.02072079,
          0.02074197],
         [0.02085561, 0.02090176, 0.02093303, ..., 0.02070913, 0.02071819,
          0.0207317 ],
         [0.02074102, 0.02083947, 0.02094038, ..., 0.02072321, 0.02071052,
          0.02071372],
         ...,
         [0.03639379, 0.03822756, 0.03722586, ..., 0.15790343, 0.12785207,
          0.09880061],
         [0.03476847, 0.03709503, 0.03786691, ..., 0.23207288, 0.18847092,
          0.14761338],
         [0.03430873, 0.03666619, 0.03686856, ..., 0.22328431, 0.2310291 ,
          0.18149282]]),
  'label': 1},
 {'sample': array([[0.02390738, 0.02383518, 0.02379105, ..., 0.02381509, 0.02382841,
          0.02388103],
         [0.02384211, 0.02378822, 0.02381906, ..., 0.02381515, 0.02383878,
          0.02383   ],
         [0.02384526, 0.02378913, 0.02380695, ..., 0.0237973 , 0.02385521,
          0.02378137],
         ...,
         [0.02497477, 0.02473888, 0.0248722 , ..., 0.

In [8]:
# Lets generate out .mat containing the gei features

In [13]:
from dataset.dataset import Dataset
from utils.image import generate_gei
from tqdm.notebook import tqdm

In [14]:
dataset = Dataset('../data/database_convertida')
body_parts=['RightHand', 
            'LeftHand',
            'UpperArmLeft',
            'UpperArmRight',
            'LowerArmLeft',
            'LowerArmRight',
            'Head']

In [15]:
nb_words = 24
dim = 100
output_dir = os.path.join(mat_root, 'gei')


database = []
for video in tqdm(dataset, total=len(dataset)):    
    if nb_words == 12 and classes[video.word] not in index12:
        continue

    gei = generate_gei(video, output_dim=(dim, dim), body_parts=body_parts)
        
    label = classes[video.word]

    data = dict()
    data['sample'] = gei
    data['label'] = label

    database.append(data)
    
dt = np.dtype([('label',(np.uint8,1)),('sample',np.float32,(dim,dim))])

arr = np.zeros((len(database), ), dtype=dt)
for idx, d in enumerate(database):
    
    arr[idx]['label'] = d['label']
    arr[idx]['sample'] = d['sample']
    
    
os.makedirs(output_dir, exist_ok=True)
mat_filename = f'database{nb_words}_gei_{dim}x{dim}.mat'
sio.savemat(os.path.join(output_dir, mat_filename), {'database': arr})

# The code above is encapsulated in the script ../tools/generate_database.py

<ipython-input-15-15fc2aa7c948>:21: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  dt = np.dtype([('label',(np.uint8,1)),('sample',np.float32,(dim,dim))])


In [16]:
mat_saved = sio.loadmat(os.path.join(output_dir, mat_filename), simplify_cells=True)
mat_saved

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Aug 28 19:39:56 2020',
 '__version__': '1.0',
 '__globals__': [],
 'database': [{'label': 24,
   'sample': array([[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)},
  {'label': 24,
   'sample': array([[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)},
  {'label': 24,
   'sample': array([[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0